# Capstone Project

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

## Data Preparation

### Data Loading

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d fedesoriano/stroke-prediction-dataset
!unzip stroke-prediction-dataset.zip

In [ ]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df.drop(['id', 'work_type'], axis=1, inplace=True)
df

### Data Cleaning

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

for col in cat_cols:
    print(f"\n=== Fitur: {col} ===")
    vc = df[col].value_counts(dropna=False)
    pct = df[col].value_counts(normalize=True, dropna=False).mul(100).round(2)
    summary = pd.DataFrame({
        'count': vc,
        'percent (%)': pct
    })
    print(summary)

In [ ]:
df = df[df['gender'] != 'Other'].copy()

#### Handling Missing Value

In [ ]:
df.isna().sum()

In [ ]:
imp_median = SimpleImputer(strategy='median')
df[['bmi']] = imp_median.fit_transform(df[['bmi']])

#### Handling Imbalance Dataset

In [ ]:
df_majority = df[df.stroke == 0]
df_minority = df[df.stroke == 1]

minority_groups = df_minority.groupby(['heart_disease', 'hypertension'])

upsampled = []
for name, group in minority_groups:
    n_samples = df_majority.shape[0] // minority_groups.ngroups
    upsampled_group = resample(
        group,
        replace=True,
        n_samples=n_samples,
        random_state=42
    )
    upsampled.append(upsampled_group)

upsampled_minority = pd.concat(upsampled)
df = pd.concat([df_majority, upsampled_minority])

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print("\nShape df:", df.shape)
df.head()

### Exploratory Data Analysis

#### Univariate Analysis

In [ ]:
numerical_features = df.select_dtypes(include='number').columns
categorical_features = df.select_dtypes(include='object').columns.to_list()

In [ ]:
for feature in categorical_features:
    count = df[feature].value_counts()
    percent = 100 * df[feature].value_counts(normalize=True)

    summary_df = pd.DataFrame({
        'jumlah sampel': count,
        'persentase': percent.round(1)
    })
    print(f"\nUnivariate summary for '{feature}':")
    print(summary_df)

    count.plot(kind='bar', title=feature)
    plt.xlabel(feature)
    plt.ylabel("Count")
    plt.show()

In [ ]:
df.hist(bins=50, figsize=(20,15))
plt.show()

#### Multivariate Analysis

In [ ]:
for col in categorical_features:
  sns.catplot(x=col, y="stroke", kind="bar", dodge=False, height = 4, aspect = 3,  data=df, palette="Set3")
  plt.title("Rata-rata 'stroke' Relatif terhadap - {}".format(col))

In [ ]:
sns.pairplot(df, diag_kind = 'kde')

In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = df[numerical_features].corr().round(2)

sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, )
plt.title("Correlation Matrix untuk Fitur Numerik ", size=20)

## Data Pre-Processing

In [ ]:
df_encoded = pd.get_dummies(df, columns=categorical_features)
df_encoded

In [ ]:
def cha2ds2vasc(row):
    score = 0
    score += int(row['heart_disease'])
    score += int(row['hypertension'])
    if row['age'] >= 75:
        score += 2
    elif row['age'] >= 65:
        score += 1
    if 'gender_Male' in row and row['gender_Male'] == 0:
        score += 1
    return score

In [ ]:
df['cha2ds2vasc_score'] = df.apply(cha2ds2vasc, axis=1)

In [ ]:
X = df_encoded.drop(columns='stroke')
y = df_encoded['stroke']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Total sample semua dataset: {len(X)}')
print(f'Total sample train dataset: {len(X_train)}')
print(f'Total sample test dataset: {len(X_test)}')

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Modelling

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.92:
            print("\nAkurasi telah mencapai > 92%!")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=128,
    validation_split=0.2,
    callbacks=[callbacks],
    verbose=2
)

## Evaluasi dan Visualisasi

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
auc_score = roc_auc_score(y_test, y_pred_prob)
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and Validation Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.title('Training and Validaion Loss')
plt.show()

In [ ]:
target_names = ['No Stroke', 'Stroke']
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names, linewidths=.5, linecolor='gray')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix Stroke')
plt.show()

y_cr = classification_report(y_test, y_pred, target_names=target_names, digits=4)
print(y_cr)

## Konversi Model

In [ ]:
model.save("model.h5")

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format=keras model.h5 tfjs_model

## Inferensi Model

In [ ]:
def infer_stroke(model, scaler, encoder, input_dict, glucose_median):
    import pandas as pd
    user_df = pd.DataFrame([input_dict])
    # Impute avg_glucose_level jika null
    if pd.isnull(user_df.loc[0, 'avg_glucose_level']):
        user_df.loc[0, 'avg_glucose_level'] = glucose_median
    user_df_encoded = pd.get_dummies(user_df)
    for col in encoder.columns:
        if col not in user_df_encoded:
            user_df_encoded[col] = 0
    user_df_encoded = user_df_encoded[encoder.columns]
    user_scaled = scaler.transform(user_df_encoded)
    proba = model.predict(user_scaled)[0][0]
    pred = int(proba >= 0.5)
    return {
        'probabilitas_stroke': f'{proba*100:.2f}%',
        'prediksi': 'Stroke' if pred == 1 else 'No Stroke'
    }

user_input = {}
user_input['gender'] = input('Jenis Kelamin (Male/Female): ')
user_input['age'] = float(input('Umur: '))
tinggi_badan = float(input('Tinggi Badan(cm): ')) / 100
berat_badan = float(input('Berat Badan(kg): '))
user_input['bmi'] = berat_badan / (tinggi_badan ** 2)
user_input['hypertension'] = int(input('Hypertension (0=tidak, 1=ya): '))
user_input['heart_disease'] = int(input('Heart Disease (0=tidak, 1=ya): '))
user_input['ever_married'] = input('Pernah Menikah? (Yes/No): ')
user_input['Residence_type'] = input('Tipe Tempat Tinggal (Urban/Rural): ')

gl_str = input('Rata-rata Glukosa (boleh dikosongi): ')
if gl_str.strip() == '':
    user_input['avg_glucose_level'] = np.nan
else:
    user_input['avg_glucose_level'] = float(gl_str)


user_input['smoking_status'] = input('Status Merokok (never smoked/smokes): ')

encoder_col = df_encoded.drop(columns='stroke')
glucose_median = df['avg_glucose_level'].median()

hasil = infer_stroke(model, scaler, encoder_col, user_input, glucose_median)
print(hasil)